### LeNet-5 실습 01

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

#### 데이터셋, 데이터로더

In [2]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.2,0.2,0.2))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.2,0.2,0.2))
])

In [3]:
train_data = CIFAR10(root="./data", train=True, download=True, transform=train_transform)
test_data = CIFAR10(root="./data", train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


#### LeNet-5 모델

In [4]:
class LeNet(nn.Module) :
    def __init__(self) :
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3) # 입력 채널, 출력 채널 수, 커널 사이즈
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3) # 입력 채널 : conv1 출력채널수, 출력 채널수: conv1 출력채널수 * pool수, 커널 사이즈
        self.fc1 = nn.Linear(64 * 6 * 6, 64) # 크기조정
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x) : 
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # 1차원으로 펼치기
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
def train_and_eval(model) : 
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # 학습 loop
    for epoch in range(5) :
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0) :
            images, labels = data
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 200 == 190 :
                print(f"Epoch [{epoch+1}/{i+1}], loss :{running_loss/200:.3f}")
                running_loss = 0.0
    print("Finished Training")

    # 모델 평가
    correct = 0 
    total = 0
    with torch.no_grad() :
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Accuracy : {acc:.2f}")

In [6]:
print("LeNet-5")
model = LeNet()
train_and_eval(model=model)

# 파라미터수
print(f"Lenet {sum(p.numel() for p in model.parameters())}")

LeNet-5
Epoch [1/191], loss :2.152
Epoch [1/391], loss :2.057
Epoch [1/591], loss :1.897
Epoch [2/191], loss :1.656
Epoch [2/391], loss :1.677
Epoch [2/591], loss :1.616
Epoch [3/191], loss :1.471
Epoch [3/391], loss :1.517
Epoch [3/591], loss :1.479
Epoch [4/191], loss :1.379
Epoch [4/391], loss :1.382
Epoch [4/591], loss :1.394
Epoch [5/191], loss :1.283
Epoch [5/391], loss :1.330
Epoch [5/591], loss :1.310
Finished Training
Accuracy : 55.14
Lenet 167562
